In [ ]:
#| default_exp common

# Common

> Base/simple components

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from itertools import product, accumulate
from enum import Enum, auto


In [ ]:
app = FastHTML(session_cookie="mysession")
rt = app.route

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

# Components

### Accordion

In [ ]:
#| export
def AccordionItem(title, content, **kwargs):
    return Details(
        Summary(
            H2(
                title,
                Icon("chevron-down", sz=18, cls="text-muted-foreground pointer-events-none size-4 shrink-0 translate-y-0.5 transition-transform duration-200 group-open:rotate-180"),
                cls="flex flex-1 items-start justify-between gap-4 py-4 text-left text-sm font-medium hover:underline"
            ),
            cls="w-full focus-visible:border-ring focus-visible:ring-ring/50 focus-visible:ring-[3px] transition-all outline-none rounded-md"
        ),
        Section(
            P(content, cls="text-sm"),
            cls="pb-4"
        ),
        cls="group border-b last:border-b-0",
        **kwargs
    )

def Accordion(*items, **kwargs):
    return Div(
        Section(*items, cls="accordion"),
        **kwargs
    )

In [ ]:
#| demo
def DemoAccordion():
    return Div(
        Accordion(
            AccordionItem("Is it accessible?", "Yes. It adheres to the WAI-ARIA design pattern."),
            AccordionItem("Is it styled?", "Yes. It comes with default styles that matches the other components' aesthetic."),
            AccordionItem("Is it animated?", "Yes. It's animated by default, but you can disable it if you prefer."),
            cls="w-96",
        ),
    cls="w-full h-96 flex items-center justify-center"
    )

In [ ]:
#| preview
p(DemoAccordion())

### Alert

In [ ]:
#| export
class AlertT(VEnum):
    default = "alert"
    destructive = "alert-destructive"

In [ ]:
#| export
def Alert(title=None, description=None, icon=None, cls=AlertT.default, **kwargs):
    content = []
    if icon: content.append(icon)
    if title: content.append(H2(title))
    if description: content.append(Section(description))
    return Div(*content, cls=cls, **kwargs)

In [ ]:
#| demo
def DemoAlert():
    return Alert(title="Warning!", icon=Icon("circle-alert"), description="Task failed. Inspect element to retry?", cls=f"{AlertT.destructive} w-96")

In [ ]:
#| preview
pw(DemoAlert())

### Avatar

In [ ]:
#| export
def Avatar(src:str, cls="", **kwargs):
    return Img(src=src, cls=f"size-8 shrink-0 object-cover rounded-full {cls}")

In [ ]:
#| demo
def DemoAvatar():
    return Avatar(src="https://github.com/hunvreus.png")

In [ ]:
#| preview
p(DemoAvatar())

### Badge

In [ ]:
#| export
class BadgeT(VEnum):
    primary = 'badge-primary'
    secondary = 'badge-secondary'
    destructive = 'badge-destructive'
    outline = 'badge-outline'

def Badge(*args, cls=BadgeT.primary, **kwargs):
    return Span(*args, cls=cls, **kwargs)

In [ ]:
#| demo
def DemoBadge():
    return Div(
        Badge("primary"),
        Badge("Secondary", cls=BadgeT.secondary),
        Badge("Destructive", cls=BadgeT.destructive),
        Badge("Outline", cls=BadgeT.outline),
        A("Link", Icon("arrow-right", height=12, width=12), cls=BadgeT.outline, href="/testing"),
    )

In [ ]:
#| preview
p(DemoBadge())

### Breadcrumb

In [ ]:
#| export
def accumulate_hrefs(links:list):
    slugs = [slugify(o) for o in links]
    return list(accumulate(slugs, lambda a, b: f"{a}/{b}", initial=""))[1:]

In [ ]:
accumulate_hrefs(["Home", "Documents", "Components", "Breadcrumb"])

['/home',
 '/home/documents',
 '/home/documents/components',
 '/home/documents/components/breadcrumb']

In [ ]:
#| export
def Breadcrumb(links:list, hrefs=None):
    if not hrefs: hrefs = accumulate_hrefs(links)
    contents = []
    for link, href in zip(links[:-1], hrefs):
        contents.append(Li(A(link, href=href, cls="hover:text-foreground transition-colors")))
        contents.append(Li(Icon("chevron-right", cls="size-3.5")))
    contents.append(Li(Span(links[-1], cls="text-foreground font-normal")))

    return Ol(
        *contents,
        cls="text-muted-foreground flex flex-wrap items-center gap-1.5 text-sm break-words sm:gap-2.5",
    )


In [ ]:
#| demo
def DemoBreadcrumb():
    return Breadcrumb(["Home", "Documents", "Components", "Breadcrumb"])

In [ ]:
#| preview
pw(
    DemoBreadcrumb()
)

NameError: name 'pw' is not defined

### Button

In [ ]:
#| export
def kebab_to_snakecase(s, strip_s=""): return s.lstrip(strip_s).replace("-", "_")

In [ ]:
kebab_to_snakecase("btn-sm-icon-primary", "btn-")

'sm_icon_primary'

In [ ]:
#| export
def mk_cls_enum(enum_name, base, variants, modifiers, sizes):
    members = {}
    for sz, mod, var in product(sizes, modifiers, variants):
        val = f"{base}{sz}{mod}{var}"
        nm = kebab_to_snakecase(val, strip_s="-btn")
        members[nm] = val
    return VEnum(enum_name, members)

In [ ]:
#| exports
ButtonT = mk_cls_enum(
    "ButtonT", 
    "btn", 
    variants=["primary", "secondary", "destructive", "outline", "ghost", "link"], 
    modifiers=["", "icon-"], 
    sizes=["-", "-sm-", "-lg-"],
)

In [ ]:
ButtonT.sm_primary

<ButtonT.sm_primary: 'btn-sm-primary'>

In [ ]:
#| export
def Button(*args, cls=ButtonT.primary, **kwargs):
    return fh.Button(*args, cls=cls, **kwargs)

In [ ]:
#| demo
def DemoButton():
    return Div(
        Button("Primary", cls=ButtonT.primary),
        Button("Secondary", cls=ButtonT.secondary),
        Button("Destructive", cls=ButtonT.destructive),
        Button("Outline", cls=ButtonT.outline),
        Button("Ghost", cls=ButtonT.ghost),
        Button("Link", cls=ButtonT.link),
        cls="flex gap-2"
    )

In [ ]:
#| preview
p(
    DemoButton()
)

### Button Group

In [ ]:
#| export
def ButtonGroup(*args, cls="", **kwargs):
    return Div(*args, role="group", cls=f"button-group {cls}", **kwargs)

In [ ]:
#| demo
def DemoButtonGroup():
    return Div(
        Button(
            Icon("arrow-left"),
            type="button",
            cls=ButtonT.icon_outline,
            aria_label="Go Back"
        ),
        ButtonGroup(
            Button("Archive", type="button", cls=ButtonT.outline),
            Button("Report", type="button", cls=ButtonT.outline),
        ),
        ButtonGroup(
            Button("Snooze", type="button", cls=ButtonT.outline),
            Button(
                Icon("ellipsis"),
                type="button",
                cls=ButtonT.icon_outline
            ),
        ),
        cls="flex w-fit items-stretch gap-2"
    )

In [ ]:
#| preview
pw(
   DemoButtonGroup()
)

### Card

In [ ]:
#| export
def Card(*contents, title=None, desc=None, footer=None, cls="", **kwargs):
    parts = []
    if title or desc: parts.append(Header(H2(title), P(desc)))
    parts.append(Section(*contents))
    if footer: parts.append(Footer(footer))
    return Div(
        *parts,
        cls=f"card {cls}",
        **kwargs
    )

In [ ]:
#| export
def CardOutline(*contents, cls="", **kwargs):
    return Div(*contents, cls=f"bg-background border border-border {cls}", **kwargs)

In [ ]:
#| demo
def DemoCard():
    return Card(
            Form(
                Div(
                    Label("Email", _for="demo-card-form-email"),
                    Input(type="email", id="demo-card-form-email"),
                    cls="grid gap-2"
                ),
                Div(
                    Div(
                        Label("Password", _for="demo-card-form-password"),
                        A("Forgot your password?", href="#", cls="ml-auto inline-block text-sm underline-offset-4 hover:underline"),
                        cls="flex items-center gap-2"
                    ),
                    Input(type="password", id="demo-card-form-password"),
                    cls="grid gap-2"
                ),
                cls="form grid gap-6"
            ),
            title="Login to your account",
            desc="Enter your details below to login to your account",
            footer=Div(
                Button("Login!", cls=ButtonT.primary + " w-full"),
                Button("Login with Google", cls=ButtonT.outline + " w-full"),
                P("Don't have an account? ", A("Sign up", href="#", cls="underline-offset-4 hover:underline"), cls="mt-4 text-center text-sm"),
                cls="flex flex-col items-center justify-center gap-2 w-full"
            ),
            cls="w-96"
        )

In [ ]:
#| preview
pw(
    DemoCard()
)

In [ ]:
#| demo
def DemoCardOutline():
    return CardOutline(
        Div(
            H3("Outline Card", cls="font-semibold"),
            P("A card with a thin outline for simplicity", cls="text-sm text-muted-foreground"),
            cls="p-6"
        ),
        cls="w-64 rounded-lg"
    )

In [ ]:
#| preview
pw(
    DemoCardOutline()
)

### Checkbox

In [ ]:
#| export
def Checkbox(*args, cls="", **kwargs):
    return Input(type="checkbox", cls=f"input {cls}", **kwargs)

In [ ]:
#| demo
def DemoCheckbox():
    return Label(Checkbox(), "Accept terms and conditions", cls="label"),

In [ ]:
#| preview
p(
    DemoCheckbox()
)

### Form

In [ ]:
#| export
def Form(*args, cls="", **kwargs):
    return fh.Form(args, cls=f"form {cls}", **kwargs)

In [ ]:
#| demo
def DemoForm():
    return Card(
        Form(
            Div(
                Label("Username", _for="demo-form-text"),
                Input(type="text", id="demo-form-text", placeholder="hunvreus"),
                P("This is your public display name.", cls="text-muted-foreground text-sm"),
                cls="grid gap-2"
            ),
            
            Div(
                Label("Email", _for="demo-form-select"),
                Select(
                    Option("m@example.com", value="bob@example.com"),
                    Option("m@google.com", value="alice@example.com"),
                    Option("m@support.com", value="john@example.com"),
                    id="demo-form-select"
                ),
                P("You can manage email addresses in your email settings.", cls="text-muted-foreground text-sm"),
                cls="grid gap-2"
            ),
            
            Div(
                Label("Bio", _for="demo-form-textarea"),
                Textarea(placeholder="I like to...", rows="3", id="demo-form-textarea"),
                P("You can @mention other users and organizations.", cls="text-muted-foreground text-sm"),
                cls="grid gap-2"
            ),
            
            Div(
                Label("Date of birth", _for="demo-form-date"),
                Input(type="date", id="demo-form-date"),
                P("Your date of birth is used to calculate your age.", cls="text-muted-foreground text-sm"),
                cls="grid gap-2"
            ),
            
            Div(
                Label("Notify me about...", _for="demo-form-radio"),
                Fieldset(
                    Label(Input(type="radio", name="demo-form-radio", value="1", checked=True), "All new messages", cls="font-normal"),
                    Label(Input(type="radio", name="demo-form-radio", value="2"), "Direct messages and mentions", cls="font-normal"),
                    Label(Input(type="radio", name="demo-form-radio", value="3"), "Nothing", cls="font-normal"),
                    id="demo-form-radio",
                    cls="grid gap-3"
                ),
                cls="flex flex-col gap-3"
            ),
            
            Section(
                H3("Email Notifications", cls="text-lg font-medium"),
                Div(
                    Div(
                        Label("Marketing emails", _for="demo-form-switch", cls="leading-normal"),
                        P("Receive emails about new products, features, and more.", cls="text-muted-foreground text-sm"),
                        cls="flex flex-col gap-0.5"
                    ),
                    Input(type="checkbox", id="demo-form-switch", role="switch"),
                    cls="gap-2 flex flex-row items-start justify-between rounded-lg border p-4 shadow-xs"
                ),
                Div(
                    Div(
                        Label("Security emails", _for="demo-form-switch-disabled", cls="leading-normal"),
                        P("Receive emails about your account security.", cls="text-muted-foreground text-sm"),
                        cls="flex flex-col gap-0.5 opacity-60"
                    ),
                    Input(type="checkbox", id="demo-form-switch-disabled", role="switch", disabled=True),
                    cls="gap-2 flex flex-row items-start justify-between rounded-lg border p-4 shadow-xs"
                ),
                cls="grid gap-4"
            ),
            
            Button("Submit", type="submit", cls="btn"),
            
            cls="grid gap-6"
        ),
    )

In [ ]:
#| preview
pw(
    DemoForm()
)

### Kbd

In [ ]:
#| export
def Kbd(*args, cls="", **kwargs):
    return fh.Kbd(*args, cls=f"kbd {cls}", **kwargs)

In [ ]:
#| demo
def DemoKbd():
    return Div(
        Span(
            Kbd("⌘"),
            Kbd("⇧"),
            Kbd("⌥"),
            Kbd("⌃"),
            cls="inline-flex items-center gap-1"
        ),
        Span(
            Kbd("Ctrl"),
            Span("+"),
            Kbd("B"),
            cls="inline-flex items-center gap-1"
        ),
        cls="flex flex-col items-center gap-4"
    )

In [ ]:
#| preview
pw(
    DemoKbd()
)

### Labels & Inputs

In [ ]:
#| export
def Input(*args, cls="", **kwargs):
    return fh.Input(*args, cls=f"input {cls}", **kwargs)

In [ ]:
p(
    Input(type="email", placeholder="Email")
)

In [ ]:
#| export
def Label(*args, cls="", **kwargs):
    return fh.Label(*args, cls=f"label {cls}", **kwargs)

In [ ]:
p(
    Label("Enter email")
)

In [ ]:
p(
    Div(
        Label("Email", _for="email"),
        Input(id="email", type="email", placeholder="Enter here...", disabled=True),
        cls="grid gap-3"
    )
)

### Pagination

In [ ]:
#| export
def mk_pagination_nums(current_page, total_pages, max_side):
    n_list = list(range(max(current_page - max_side, 1), min(current_page + max_side + 1, total_pages+1)))
    if n_list[1] != 2: n_list[1] = "..."
    if n_list[-2] != total_pages-1: n_list[-2] = "..."
    n_list[0] = 1
    n_list[-1] = total_pages
    return n_list

In [ ]:
print(mk_pagination_nums(10, 30, 6))
print(mk_pagination_nums(2, 30, 5))

[1, '...', 6, 7, 8, 9, 10, 11, 12, 13, 14, '...', 30]
[1, 2, 3, 4, 5, '...', 30]


In [ ]:
#| export
def Pagination(current_page, total_pages, max_side=5, href_pattern="/page/{n}"):
    contents = [Li(A(Icon("chevron-left"), " Previous", href=href_pattern.format(n=max(current_page-1, 1)), cls=ButtonT.ghost))]
    for n in mk_pagination_nums(current_page, total_pages, max_side):
        if n == "...":
            contents.append(Li(Div(Icon("ellipsis", cls="size-4 shrink-0"), cls="size-9 flex items-center justify-center")))
        else:
            cls = ButtonT.icon_outline if n == current_page else ButtonT.icon_ghost
            contents.append(Li(A(n, href=href_pattern.format(n=n), cls=cls)))
    contents.append(Li(A("Next ", Icon("chevron-right"), href=href_pattern.format(n=min(current_page+1, total_pages)), cls=ButtonT.ghost)))
    return Nav(Ul(*contents, cls="flex flex-row items-center gap-1"), role="navigation", aria_label="pagination")

In [ ]:
pw(    
    Pagination(2, 20)
)

### Progress

In [ ]:
#| export
def Progress(value=0, cls="", **kwargs):
    return Div(
        Div(style=f"width: {value}%", cls="bg-primary h-full w-full flex-1 transition-all"),
        cls=f"bg-primary/20 relative h-2 w-full overflow-hidden rounded-full {cls}",
        **kwargs
    )

In [ ]:
pw(Progress(value=50))

### Radio Group

In [ ]:
#| export
def Radio(name, value=None, **kwargs):
    if not value: value = name.lower()
    return Label(Input(type="radio", name="radio-group", value=value, **kwargs), name, cls="label")

In [ ]:
p(Radio("test"))

In [ ]:
#| export
def RadioGroup(*radios, cls="", **kwargs):
    return Fieldset(*radios, cls=f"grid gap-3 {cls}", **kwargs)

In [ ]:
pw(
    RadioGroup(
        Radio("Default"),
        Radio("Comfortable", checked=True),
        Radio("Compact"),
    )
)

### Skeleton

In [ ]:
#| export
def SkeletonCircle(cls="size-10", **kwargs):
    return Div(cls=f"bg-accent animate-pulse rounded-full shrink-0 {cls}", **kwargs)

def SkeletonLine(cls="w-full", **kwargs):
    return Div(cls=f"bg-accent animate-pulse rounded-md h-4 {cls}", **kwargs)

In [ ]:
#| export
def SkeletonSquare(cls="size-10", **kwargs):
    return Div(cls=f"bg-accent animate-pulse rounded-md shrink-0 {cls}", **kwargs)

In [ ]:
pw(
    Div(
        SkeletonCircle(),
        Div(SkeletonLine(cls="w-[150px]"), SkeletonLine(cls="w-[100px]"), cls="grid gap-2"),
        cls="flex items-center gap-4"
    )
)

In [ ]:
pw(
    Div(
        SkeletonSquare(),
        SkeletonSquare(),
        cls="flex gap-4"
    )
)

### Slider

Note: slider requires some JS to dynamically update the val. To make things easier I've defined the `updateSlider` func in the headers, and then each Slider component includes a script that sets up a listener for that.

In [ ]:
#| export
def Slider(min="0", max="100", value="50", cls="", **kwargs):
    return Div(
        Input(type="range", cls=f"input {cls}", min=min, max=max, value=value, **kwargs),
        Script("""
            const el = document.currentScript.previousElementSibling;
            updateSlider(el);
            el.addEventListener('input', () => updateSlider(el));
        """)
    )

In [ ]:
pw(
    Slider(cls="w-96")
)

### Spinner

In [ ]:
#| export
def Spinner(cls="", **kwargs):
    return Icon("loader-circle", cls=f"animate-spin {cls}", **kwargs)

In [ ]:
pw(
    Div(
        Div(Spinner(), cls="flex shrink-0 items-center justify-center [&_svg]:pointer-events-none [&_svg]:size-4"),
        Div(H3("Processing payment...", cls="flex w-fit items-center gap-2 text-sm leading-snug font-medium line-clamp-1"), cls="flex flex-1 flex-col gap-1"),
        Div(P("$100.00", cls="text-muted-foreground line-clamp-2 text-sm leading-normal font-normal text-balance [&>a:hover]:text-primary [&>a]:underline [&>a]:underline-offset-4"), cls="flex flex-col gap-1 flex-none text-center"),
        cls="group/item flex items-center border text-sm rounded-md transition-colors flex-wrap outline-none focus-visible:border-ring focus-visible:ring-ring/50 focus-visible:ring-[3px] border-transparent p-4 gap-4 bg-card"
    )
)

### Switch

In [ ]:
#| export
def Switch(*args, **kwargs):
    return Input(*args, type="checkbox", role="switch", cls="input", **kwargs)

In [ ]:
p(
    Switch()
)

### Table

In [ ]:
#| export
def TableFromDict(table_dict:dict, caption=None):
    heads = [Th(o) for o in table_dict.keys()]
    rows = [Tr(*[Td(val) for val in row]) for row in zip(*table_dict.values())]
    return Table(
        Caption(caption),
        Thead(Tr(*heads)),
        Tbody(*rows),
        cls="table",
    )

In [ ]:
pw(
    Div(
        TableFromDict(
            {
                "Invoice": ["INV001", "INV002", "INV003", "INV004", "INV005", "INV006", "INV007"],
                "Status": ["Paid", "Pending", "Unpaid", "Paid", "Paid", "Pending", "Unpaid"],
                "Method": ["Credit Card", "PayPal", "Bank Transfer", "Paypal", "Credit Card", "Bank Transfer", "Credit Card"],
                "Amount": ["$250.00", "$150.00", "$350.00", "$450.00", "$550.00", "$200.00", "$300.00"]
            },
            caption="A list of your recent invoices"
        ),
        cls="w-xl overflow-x-auto",
    )
)

### Textarea

In [ ]:
#| export
def Textarea(*args, placeholder="", cls="", invalid="false", **kwargs):
    return fh.Textarea(*args, placeholder=placeholder, cls=f"textarea {cls}", aria_invalid=invalid, **kwargs)

In [ ]:
pw(
    Textarea(placeholder="testing", cls="w-100 m-2"),
    Textarea(placeholder="testing", invalid="true", cls="w-100 m-2"),
    Textarea(placeholder="testing", disabled=True, cls="w-100 m-2"),
)

### Theme Switcher

In [ ]:
#| export
def ThemeSwitcher(cls="", **kwargs):
    return Button(
        Span(Icon("sun"), cls="hidden dark:block"),
        Span(Icon("moon"), cls="block dark:hidden"),
        type="button",
        data_side="bottom", onclick="document.dispatchEvent(new CustomEvent('basecoat:theme'))",
        cls=ButtonT.icon_outline + f"size-8 {cls}"
    )

In [ ]:
pw(
    ThemeSwitcher()
)

### Toast

In [ ]:
#| export
def ToastCloseButton(text="Dismiss"):
    return Button(text, data_toast_action=True, cls=ButtonT.primary)

In [ ]:
#| export
def Toaster(data_align="end"):
    "data_align accepts: ['start', 'center', end']"
    return Div(id="toaster", cls="toaster", data_align=data_align)

In [ ]:
#| export
def Toast(title, desc, icon=Icon("circle-check"), footer=ToastCloseButton(), cls="", role="status", data_category="success", aria_atomic="true", aria_hidden="false", **kwargs):
    return Div(
        Div(icon, Section(H2(title), P(desc)), Footer(footer), cls="toast-content"),
        cls=f"toast {cls}",
        role=role,
        aria_atomic=aria_atomic,
        aria_hidden=aria_hidden,
        data_category=data_category,
        **kwargs
    )

In [ ]:
@rt
def toast_example():
    return Toast("Success", "Toast send from backend!")

In [ ]:

@rt
def toast_example_page():
    return Window(
        Toaster(),
        Button(
            "Toast from backend (with HTMX)",
            cls=ButtonT.outline,
            hx_get=toast_example,
            hx_target="#toaster",
            hx_swap="beforeend"
        )
    )

In [ ]:
pw(
    toast_example_page()
)

### Tooltip

Note: tooltip is not a component. You can add tooltips to any other element by adding a data_tooltip attribute.

In [ ]:
#| export
def tooltip_kwargs(
    text:str,      # The text to display
    side="top",    # either 'top', 'buttom', 'left', or 'right'
    align="center" # either 'start', 'center', or 'end'
):
    return dict(data_tooltip=text, data_side=side, data_align=align) 

In [ ]:
pw(
    Button("Hover", cls=ButtonT.outline, **tooltip_kwargs("Add to library")),
    Button("Hover", cls=ButtonT.outline, **tooltip_kwargs("Add to library", side="left", align="end")),
    Button("Hover", cls=ButtonT.outline, **tooltip_kwargs("Add to library", side="right"))
)